In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import base as b
import hvplot.pandas  # noqa
from functools import cache
from logger_settings import logger
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE


TEST_START = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-02-15", "%Y-%m-%d")

train_dates, test_dates = ut.get_date_range(start_date=TEST_START, end_date=TEST_END, symbol=SYMBOL, interval=INTERVAL, exchange=EXCHANGE)
train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

train_dates['call_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_CALL, price=30, exchange=EXCHANGE), axis=1)
train_dates['put_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=30, exchange=EXCHANGE), axis=1)

# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

/home/nitin/nse-reports-trade/utils.py:39: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  scrip_df = pd.read_csv(file_path)


Loading BokehJS ...

In [2]:
train_dates

,expiry,call_strike,put_strike
trade_date,,,
2024-01-01,2024-01-04,22050,21350
2024-01-02,2024-01-04,22000,21500
2024-01-05,2024-01-11,22050,21400
2024-01-08,2024-01-11,21950,21450
2024-01-11,2024-01-11,21800,21600
2024-01-12,2024-01-18,22100,21300
2024-01-16,2024-01-18,22300,21700
2024-01-17,2024-01-18,22100,21500
2024-01-18,2024-01-18,21550,21300


In [8]:
# train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=30), axis=1)
ut.get_strike_price_by_price(symbol=SYMBOL, expiry=train_dates.iloc[0].expiry, timestamp=train_dates.iloc[0].name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=30)

IndexError: single positional indexer is out-of-bounds

In [2]:
nifty_candles = ut.get_data(symbol=SYMBOL, date=test_date, interval=INTERVAL, exchange=EXCHANGE)
atm_strike = 21700
otm_call_strike = 22100
otm_put_strike = 21400
expiry = ut.find_nclosest_expiry(SYMBOL, test_date, 1)

call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_call_strike, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_put_strike, option_type=OPTION_TYPE_PUT)

In [67]:
ut.create_candlestick_plot(call_df)
ut.create_candlestick_plot(put_df)

GridPlot(id='p1836', ...)

In [14]:
settings = {
    "VOLUME_QUANTILE_THRESHOLD": 10 / 100, # Choose least this % volume from the given volumes
    "CANDLE_LENTH": 20,
    "MIN_GREEN_CANDLE_LENGTH": 3, # Minimum candles to be considered as pattern
    "RED_CANDLE_RATIO": 0.65,
    "GREEN_THRESHOLD": 0.1, # For candle to be considered green, percentage of standard deviation
    # Order Settings
    "quantity": 25,
    "TARGET_PC": 0.05,
    "STOPLOSS_PC": 0.03,
}

from typing import Dict


class PeakStrategy(b.Strategy):
    def __init__(self, instrument: "Instrument", settings: Dict):
        super().__init__(instrument, settings)
        self.call_ticks: pd.DataFrame | None = None
        self.put_ticks: pd.DataFrame | None = None
        self.om = b.OrderManager()

    def calculate_data(self):
        self.put_ticks['qv'] = self.put_ticks['volume'].rolling(window=self.CANDLE_LENTH).quantile(self.VOLUME_QUANTILE_THRESHOLD)
        self.put_ticks['qv_ratio'] = self.put_ticks.volume / self.put_ticks.qv
        self.put_ticks['qv_qualify'] = self.put_ticks.qv_ratio > 5
        self.put_ticks['prev_ratio'] = self.put_ticks.volume / self.put_ticks.volume.shift(1)
        self.put_ticks['prev_qv_qualify'] = self.put_ticks.qv_qualify.shift(1)
        self.put_ticks['red_qualify'] = self.put_ticks.prev_qv_qualify & (self.put_ticks.prev_ratio < self.RED_CANDLE_RATIO)
        self.put_ticks['cdiff'] = self.put_ticks.close.diff()
        self.put_ticks['is_green'] = self.put_ticks.cdiff > 0

    def is_volume_match(self) -> bool:
        '''
        1. Must have high volume on the last green candle
        2. Must have low volume on the last red candle
        '''
        return self.put_ticks.iloc[-1].red_qualify

    def is_price_match(self):
        '''
        1. Price must be increasing until the last candle
        2. There must not be big gaps in the candles, that could mean that it's going to go further up
        '''
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        green_length = last_n_ticks.loc[last_n_ticks.is_green].shape[0]
        if green_length < self.MIN_GREEN_CANDLE_LENGTH:
            last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-2:-1]
        if last_tick.is_green:
            return False
        if green_length < self.MIN_GREEN_CANDLE_LENGTH - 1:
            return False
        elif green_length == self.MIN_GREEN_CANDLE_LENGTH:
            return True
        red_diff = last_n_ticks.loc[~last_n_ticks.is_green].cdiff.abs().sum()
        if red_diff < last_n_ticks.loc[last_n_ticks.cdiff > 0].cdiff.sum() * self.GREEN_THRESHOLD:
            return True
        return False

    def entry_conditions(self) -> bool:
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        is_price_match = self.is_price_match()
        is_volume_match = self.is_volume_match()
        if not is_volume_match:
            return False
        if not is_price_match:
            return False
        print(f"matched at: {self.put_ticks.iloc[-1].timestamp}")
        return True

    def exit_conditions(self) -> bool:
        last_price = self.call_ticks.iloc[-1].close
        for order in self.om.orders:
            if ((last_price - order.limit_price) / order.limit_price) >= self.TARGET_PC:
                return True
            if ((order.limit_price - last_price) / order.limit_price) >= self.STOPLOSS_PC:
                return True
        return False

    def _process_tick(self, call_tick: Dict, put_tick: Dict) -> None:
        call_tick_df = pd.DataFrame(
            call_tick, index=[len(self.call_ticks) if self.call_ticks is not None else 0]
        )
        self.call_ticks = pd.concat([self.call_ticks, call_tick_df], ignore_index=True)

        put_tick_df = pd.DataFrame(
            put_tick, index=[len(self.put_ticks) if self.put_ticks is not None else 0]
        )
        self.put_ticks = pd.concat([self.put_ticks, put_tick_df], ignore_index=True)

    def next(self, call_tick: Dict, put_tick: Dict):
        self._process_tick(call_tick, put_tick)
        self.calculate_data()
        if not self.om.has_intrade_orders() and self.entry_conditions():
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.call_ticks.iloc[-1].close, created_at=self.call_ticks.iloc[-1].name, quantity=self.quantity, exchange_order_id=None)
            self.om.place_order(order)
        if self.om.has_intrade_orders() and self.exit_conditions():
            self.om.square_off_all_orders(index=self.call_ticks.iloc[-1].name, last_price=self.call_ticks.iloc[-1].close)



instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

ps = PeakStrategy(instrument=instrument, settings=settings)
# for i in range(2000):
# for i in range(1293, 3221):
for i in range(put_df.shape[0]):
    cdict = call_df.iloc[i].to_dict()
    cdict['timestamp'] = call_df.iloc[i].name
    pdict = put_df.iloc[i].to_dict()
    pdict['timestamp'] = put_df.iloc[i].name
    # cdict, pdict = pdict, cdict
    ps.next(cdict, pdict)


matched at: 2024-01-01 15:24:00


In [15]:
ps.om.closed_orders

[buy, at:369, b:10.85, sqat:370, s:10.4, pnl:-4.147465437788012]

In [31]:
pd.set_option('display.max_rows', 200)
ps.put_ticks.head(200)

,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,timestamp,qv,qv_ratio,qv_qualify,prev_ratio,prev_qv_qualify,red_qualify,cdiff,is_green,is_close_green
0,17.05,19.35,7.35,17.80,235500.0,2881650.0,17.78,17.65,17.75,2024-01-01 09:15:00,NaN,NaN,False,NaN,NaN,False,NaN,False,False
1,17.60,17.95,16.55,17.75,192000.0,2881650.0,17.56,17.70,17.75,2024-01-01 09:16:00,NaN,NaN,False,0.815287,False,False,-0.05,False,True
2,17.50,17.50,16.35,16.75,274150.0,2930050.0,17.27,16.75,16.80,2024-01-01 09:17:00,NaN,NaN,False,1.427865,False,False,-1.00,False,True
3,16.40,16.70,15.95,16.15,280700.0,2930050.0,17.01,16.10,16.20,2024-01-01 09:18:00,NaN,NaN,False,1.023892,False,False,-0.60,False,True
4,16.00,16.55,15.60,16.45,123000.0,2930050.0,16.89,16.40,16.45,2024-01-01 09:19:00,NaN,NaN,False,0.438190,False,False,0.30,True,True
5,16.30,16.60,15.70,16.40,168700.0,3087150.0,16.78,16.35,16.45,2024-01-01 09:20:00,NaN,NaN,False,1.371545,False,False,-0.05,False,True
6,16.55,16.55,16.05,16.40,119300.0,3087150.0,16.74,16.35,16.40,2024-01-01 09:21:00,NaN,NaN,False,0.707172,False,False,0.00,False,True
7,16.30,16.50,16.00,16.45,90550.0,3087150.0,16.71,16.35,16.45,2024-01-01 09:22:00,NaN,NaN,False,0.759011,False,False,0.05,True,True
8,16.30,16.55,16.15,16.30,57400.0,3110900.0,16.70,16.20,16.30,2024-01-01 09:23:00,NaN,NaN,False,0.633904,False,False,-0.15,False,True
9,16.35,16.35,15.80,15.80,59050.0,3110900.0,16.67,15.80,15.85,2024-01-01 09:24:00,NaN,NaN,False,1.028746,False,False,-0.50,False,True
